# OpenStreetMap Data Case Study
### Portland, Oregon
#### source: https://mapzen.com/data/metro-extracts/metro/portland_oregon/

In [4]:
import xml.etree.cElementTree as cET 
import re as re
from collections import defaultdict
import pprint as pprint

In [5]:
example_file = "sample.osm"

In [6]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
address_re = re.compile(r'^addr\:')
street_re = re.compile(r'^street')

In [ ]:
# street names -- full
expected = ["Alley", "Anex", "Arcade", "Avenue"]

# mapping abbreviated street names to full street names
# link: http://pe.usps.gov/text/pub28/28apc_002.htm
mapping = {"Aly" : "Alley",
           "Anx" : "Anex",
           "Arc" : "Arcade",
           "Ave" : "Avenue",
           
            }

In [7]:
def count_tags(example_file):
    tags = {}
    for event, elem in cET.iterparse(filename):
        if tags.has_key(elem.tag) == True:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

def unique_users(example_file):
    users = set()
    for _, element in cET.iterparse(filename):
        if 'user' in element.attrib:
            users.update([element.attrib['user']])
    return users

In [8]:
count_tags(example_file)

{'member': 6650,
 'nd': 734588,
 'node': 638587,
 'osm': 1,
 'relation': 612,
 'tag': 487534,
 'way': 82289}

In [13]:
print len(unique_users(example_file))

742
